In [9]:
import numpy as np
import pandas as pd
import requests
import tqdm
import time
import random

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep

In [11]:
url = "https://www.autocasion.com/coches-ocasion"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Crear una sesión
session = requests.Session()

# Hacer la solicitud con la sesión
response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

200


In [119]:
total_coches = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.',''))

total_paginas = int(total_coches/len(anuncios)) + (1 if (total_coches/len(anuncios))%1 else 0)

In [25]:
anuncios = soup.find_all("article", class_="anuncio")

anuncio = soup.find("article", class_="anuncio")
anuncio.find("a")['href']

In [129]:
def links_paginas():
    url = "https://www.autocasion.com/coches-ocasion"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
    session = requests.Session()
    soup = BeautifulSoup(response.content, "html.parser")
    url_pagina = 'https://www.autocasion.com/coches-ocasion?page='
    lista_urls = []
    for pagina in range(0, total_paginas +1):
        lista_urls.append(f'{url_pagina}{pagina}')
        
    return lista_urls

In [137]:
lista_urls = links_paginas()
lista_urls[-1]

'https://www.autocasion.com/coches-ocasion?page=6538'

In [32]:
# sacar los links de todos los anuncios, guardarlos en una lista y luego sacar los datos de cada coche

for coche in anuncios:
    venta = coche.find("a")['href']
    print(f'{url}{venta}')

https://www.autocasion.com/coches-ocasion/coches-segunda-mano/mercedes-benz-clase-a-ocasion/a-180-166b59d7759ecc-ref14036367
https://www.autocasion.com/coches-ocasion/coches-segunda-mano/bmw-serie-5-ocasion/520da-xdrive-3-ref11961816
https://www.autocasion.com/coches-ocasion/coches-segunda-mano/bmw-x4-ocasion/x4-xdrive-30da-10-ref13069571
https://www.autocasion.com/coches-ocasion/coches-segunda-mano/bmw-x1-ocasion/x1-sdrive-18da-5-ref14614142
https://www.autocasion.com/coches-ocasion/coches-segunda-mano/mercedes-benz-eqc-ocasion/eqc-400-4matic-ref8610727
https://www.autocasion.com/coches-ocasion/coches-segunda-mano/fiat-tipo-ocasion/tipo-1-4-pop-95-ref14401473
https://www.autocasion.com/coches-ocasion/coches-segunda-mano/bmw-i8-ocasion/i8-ref14199673
https://www.autocasion.com/coches-ocasion/coches-segunda-mano/fiat-500-ocasion/500-1-0-hybrid-connect-52kw-ref14593999
https://www.autocasion.com/coches-ocasion/coches-segunda-mano/audi-q3-ocasion/q3-sportback-45-tfsi-black-line-quattro-re